This is the exam, first create the database in python

In [2]:
import sqlite3
import pandas as pd

In [5]:
import sqlite3

# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('BookStoreDB.db')
cursor = conn.cursor()

# Create the Book table
cursor.execute('''
CREATE TABLE Book (
    BookID INTEGER PRIMARY KEY,
    Title TEXT,
    Author TEXT,
    Publisher TEXT,
    Year INTEGER,
    Price REAL
)
''')

# Create the Sold table
cursor.execute('''
CREATE TABLE Sold (
    Name TEXT,
    BookID INTEGER,
    Quantity INTEGER,
    PRIMARY KEY (Name, BookID),
    FOREIGN KEY (BookID) REFERENCES Book(BookID)
)
''')

# Create the Bookstore table
cursor.execute('''
CREATE TABLE Bookstore (
    Name TEXT PRIMARY KEY,
    Address TEXT,
    City TEXT
)
''')

# Insert data into the Book table
books = [
    (1, 'The Great Gatsby', 'F. Scott Fitzgerald', 'Scribner', 2001, 10.99),
    (2, 'To Kill a Mockingbird', 'Harper Lee', 'Harper & Brothers', 2009, 12.50),
    (3, 'The Baby Boomers', 'George Orwell', 'Secker & Warburg', 2020, 8.75),
    (4, 'Pride and Prejudice', 'Jane Austen', 'T. Egerton', 2010, 9.99),
    (5, 'Harry Potter and the Sorcerer\'s Stone', 'J.K. Rowling', 'Bloomsbury', 1997, 14.95)
]
cursor.executemany('INSERT INTO Book VALUES (?, ?, ?, ?, ?, ?)', books)

# Insert data into the Sold table
sold = [
    ('Book Haven', 1, 500),
    ('Reading Nook', 2, 300),
    ('Novel Corner', 3, 250),
    ('Book Haven', 4, 870),
    ('Reading Nook', 1, 200),
    ('Novel Corner', 5, 630)
]
cursor.executemany('INSERT INTO Sold VALUES (?, ?, ?)', sold)

# Insert data into the Bookstore table
bookstores = [
    ('Book Haven', '123 Main St', 'New York'),
    ('Reading Nook', '456 Oak Ave', 'Los Angeles'),
    ('Novel Corner', '789 Elm St', 'Chicago')
]
cursor.executemany('INSERT INTO Bookstore VALUES (?, ?, ?)', bookstores)

# Commit the changes and close the connection
conn.commit()
conn.close()



OperationalError: table Book already exists

In [ ]:
SELECT DISTINCT Bookstore.Name
FROM Bookstore
JOIN Sold ON Bookstore.Name = Sold.Name
JOIN Book ON Sold.BookID = Book.BookID
WHERE Bookstore.City = 'Chicago'
  AND Book.Year BETWEEN 2000 AND 2020;


Find the names of the bookstores that are in Chicago and have sold some books published between the years 2000 and 2020

In [15]:
conn = sqlite3.connect('BookStoreDB.db')
cursor = conn.cursor()

query = '''
SELECT DISTINCT Bookstore.Name
FROM Bookstore
JOIN Sold ON Bookstore.Name = Sold.Name
JOIN Book ON Sold.BookID = Book.BookID
WHERE Bookstore.City = 'Chicago'
  AND Book.Year BETWEEN 2000 AND 2020;
'''
result_df = pd.read_sql_query(query, conn)


conn.close()

print(result_df.to_string(index=True))

           Name
0  Novel Corner


2. Find the names of the bookstores that have sold every book that is sold by the bookstore named "Book Haven"

In [23]:
conn = sqlite3.connect('BookStoreDB.db')
cursor = conn.cursor()

query = '''
SELECT DISTINCT(Sold.Name)
FROM Sold 
WHERE Sold.BookID IN
    (SELECT Book.BookID
    FROM Book
    JOIN Sold ON Sold.BookID = Book.BookID 
    WHERE Name = 'Book Haven')
      
'''
result_df = pd.read_sql_query(query, conn)


conn.close()

print(result_df.to_string(index=True))

           Name
0    Book Haven
1  Reading Nook


3. Find the names of the bookstores that have sold at most 500 different books (of any quantity) published after the year 2000

In [25]:
conn = sqlite3.connect('BookStoreDB.db')
cursor = conn.cursor()

query = '''
SELECT DISTINCT(Name)
FROM Book
JOIN Sold ON Sold.BookID = Book.BookID
WHERE Quantity > 499 AND Year > 2000 
      
'''
result_df = pd.read_sql_query(query, conn)


conn.close()

print(result_df.to_string(index=True))

         Name
0  Book Haven


4. Find the names of the New York bookstores that have sold more books than the bookstore named "Novel Corner".

In [27]:
conn = sqlite3.connect('BookStoreDB.db')
cursor = conn.cursor()

query = '''

SELECT Sold.Name 
FROM Sold 
JOIN Bookstore ON Bookstore.Name = Sold.Name 
WHERE City = 'New York' AND Quantity < (SELECT SUM(Quantity) FROM Sold WHERE Name = 'Novel Corner')
GROUP BY Sold.Name
      
'''
result_df = pd.read_sql_query(query, conn)


conn.close()

print(result_df.to_string(index=True))

         Name
0  Book Haven
